In [1]:
!pip install tensorflow-gpu==1.15.3

     |████████████████████████████████| 411.0MB 37kB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 3.8MB 49.7MB/s 
     |████████████████████████████████| 512kB 56.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=a7db4ae7cda931bfdb9eee79b0a527b72eb283fc4997e120813a78750900262d
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorboard~=2.4, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorflow-estimator<2.5.0,>=2.4.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing instal

In [7]:
!pip install Keras==2.1.5

     |████████████████████████████████| 337kB 13.2MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [2]:
!git clone https://github.com/qqwweee/keras-yolo3

Cloning into 'keras-yolo3'...
remote: Enumerating objects: 144, done.
remote: Total 144 (delta 0), reused 0 (delta 0), pack-reused 144
Receiving objects: 100% (144/144), 151.08 KiB | 747.00 KiB/s, done.
Resolving deltas: 100% (65/65), done.


In [3]:
%cd keras-yolo3

/content/keras-yolo3


In [8]:
!wget https://pjreddie.com/media/files/yolov3.weights yolov3.weights
!python convert.py yolov3.cfg yolov3.weights model_data/yolo_weights.h5

--2021-02-24 06:51:15--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights.1’

yolov3.weights.1    100%[===================>] 236.52M  18.8MB/s    in 21s     

2021-02-24 06:51:37 (11.4 MB/s) - ‘yolov3.weights.1’ saved [248007048/248007048]

--2021-02-24 06:51:37--  http://yolov3.weights/
Resolving yolov3.weights (yolov3.weights)... failed: Name or service not known.
wget: unable to resolve host address ‘yolov3.weights’
FINISHED --2021-02-24 06:51:37--
Total wall clock time: 22s
Downloaded: 1 files, 237M in 21s (11.4 MB/s)
Using TensorFlow backend.
Loading weights.
Weights Header:  0 2 0 [32013312]
Parsing Darknet config.
Creating Keras model.


Parsing section net_0
Parsing section convolutional_0
conv2d bn leaky (3, 3, 3, 32)




2021

In [5]:
!git clone https://github.com/worldstar/dataset.git

Cloning into 'dataset'...
remote: Enumerating objects: 713, done.
remote: Total 713 (delta 0), reused 0 (delta 0), pack-reused 713
Receiving objects: 100% (713/713), 7.26 MiB | 2.62 MiB/s, done.
Resolving deltas: 100% (341/341), done.


In [19]:
import xml.etree.ElementTree as ET
import os
from os import getcwd
import sys

def generateAnnotations(path, imagePath, writePath, voc_classes):
    # path = sys.argv[1] #"./Data/Annotations/"
    # imagePath = sys.argv[2]#"./Data/JPEGImages/"
    # writePath = sys.argv[3]#"./model_data/train.txt"
    deputyFileName = "jpg"
    # classes = ["bicycle","car","cat","dog","person"]
    fr = open(voc_classes, 'r')#"model_data/voc_classes.txt"
    classes = fr.read().split("\n")
    fr.close()

    fw = open(writePath, "w")
    for fileName in os.listdir(path):
        if fileName in ".gitignore":
            continue
        print("readFile:",(path+fileName))
        convertResult = convert_annotation((path+fileName),classes,imagePath,deputyFileName)
        fw.write(convertResult)
    fw.close()

def dataUs(infos):
    return infos.split(".")[0]

def convert_annotation(path,classes,imagePath,deputyFileName): 
    try: 
        xmlFile = open(path) 
    except:
        xmlFile = open(path,encoding="utf-8")   
    xmlTree = ET.parse(xmlFile)
    xmlRoot = xmlTree.getroot()
    width,height,depth = -1,-1,-1
    hasClass = False
    result = ""
    for xmlObj in xmlRoot.iter('size'):
        width = xmlObj.find('width').text.replace(" ", "").replace("\t", "").replace("\n", "")
        height = xmlObj.find('height').text.replace(" ", "").replace("\t", "").replace("\n", "").replace(" ", "").replace("\t", "").replace("\n", "")
        depth = xmlObj.find('depth').text.replace(" ", "").replace("\t", "").replace("\n", "")
        # print(width,height,depth)
    for xmlObj in xmlRoot.iter('object'):
        name = xmlObj.find('name').text.replace(" ", "").replace("\t", "").replace("\n", "")
        isClass = False
        classNum = -1
        for i in range(0,len(classes),1):
            if(name == classes[i]):
                isClass = True
                hasClass = True
                classNum = i
        if(isClass):
            xmin , ymin , xmax , ymax = -1 , -1 , -1 , -1
            for xmlObj2 in xmlObj.iter('bndbox'):
                xmin = int(dataUs(xmlObj2.find('xmin').text.replace(" ", "").replace("\t", "").replace("\n", "").replace(" ", "").replace("\t", "").replace("\n", "")))
                ymin = int(dataUs(xmlObj2.find('ymin').text.replace(" ", "").replace("\t", "").replace("\n", "")))
                xmax = int(dataUs(xmlObj2.find('xmax').text.replace(" ", "").replace("\t", "").replace("\n", "")))
                ymax = int(dataUs(xmlObj2.find('ymax').text.replace(" ", "").replace("\t", "").replace("\n", "")))
            result += " %s,%s,%s,%s,%d"%(xmin,ymin,xmax,ymax,classNum)
    if(hasClass):
        FileName = os.path.basename(path)
        FileName = os.path.splitext(FileName)[0]
        FileName = FileName+"."+deputyFileName
        result = "%s%s\n"%((imagePath + FileName),result)
    return result


In [27]:
fw = open('model_data/voc_classes.txt', "w")
fw.write('RBC')
fw.close()

In [28]:
generateAnnotations('dataset/Annotations/', 'dataset/JPEGImages/', 'train.txt', 'model_data/voc_classes.txt')

readFile: dataset/Annotations/BloodImage_00372.xml
readFile: dataset/Annotations/BloodImage_00020.xml
readFile: dataset/Annotations/BloodImage_00057.xml
readFile: dataset/Annotations/BloodImage_00177.xml
readFile: dataset/Annotations/BloodImage_00049.xml
readFile: dataset/Annotations/BloodImage_00391.xml
readFile: dataset/Annotations/BloodImage_00287.xml
readFile: dataset/Annotations/BloodImage_00145.xml
readFile: dataset/Annotations/BloodImage_00013.xml
readFile: dataset/Annotations/BloodImage_00257.xml
readFile: dataset/Annotations/BloodImage_00159.xml
readFile: dataset/Annotations/BloodImage_00255.xml
readFile: dataset/Annotations/BloodImage_00335.xml
readFile: dataset/Annotations/BloodImage_00408.xml
readFile: dataset/Annotations/BloodImage_00048.xml
readFile: dataset/Annotations/BloodImage_00388.xml
readFile: dataset/Annotations/BloodImage_00029.xml
readFile: dataset/Annotations/BloodImage_00069.xml
readFile: dataset/Annotations/BloodImage_00371.xml
readFile: dataset/Annotations/B

In [29]:
!python train.py

Using TensorFlow backend.








2021-02-24 07:36:03.634504: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2021-02-24 07:36:03.639106: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000140000 Hz
2021-02-24 07:36:03.639299: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5616b71c2a00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-02-24 07:36:03.639328: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-02-24 07:36:03.641045: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-02-24 07:36:03.747417: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning N